In [ ]:
import time
import datetime
import pandas as pd
import os
import openai

from entities import whatsapp_comands as wc
from models import classes


openai.api_key = os.getenv('OPENAI_API_KEY')
base_path = f'ct_files/input/BQ.xlsx'
output_path = f'ct_files/output/OQ-{datetime.date.today()}.xlsx'

In [ ]:
if __name__ == '__main__':
    wc.star_robot()

In [ ]:
questions_df = pd.read_excel(base_path)

In [28]:
start = 0 
interactions = []

for index, row in questions_df.iterrows():
    if index >= start:
        questions = row['Questions']
        expected_answer = row['Expected_Answer']
        
        wc.send_message(questions)
        message_sent = questions
        actual_message = ''
        start_time = time.time()
        
        while True: 
            actual_message = wc.get_last_message()
            time.sleep(1)
            if actual_message != message_sent and actual_message != 'Espere um minuto...':
                end_time = time.time()
                elapsed_time = end_time - start_time
                elapsed_time_formatted = "{:.2f}".format(elapsed_time)
                wc.open_read_more()
                time.sleep(2)
                
                interactions.append(classes.Interaction(questions, expected_answer, actual_message, elapsed_time_formatted, ''))
                break

interactions_df = pd.DataFrame([{
    'Question': interaction.question,
    'Expected_Message': interaction.expected_message,
    'Answer': interaction.answer,
    'Answer_Time': interaction.answer_time,
    'Simalirity': interaction.simalirity
} for interaction in interactions])
interactions_df.to_excel(output_path, index=False)

In [ ]:
def validator(text1, text2, model="gpt-3.5-turbo"):
    prompt = f"""
        # Mission
        Your sole mission is to compare two distinct texts and provide a numerical similarity score ranging from 0 to 1, reflecting the degree of similarity between these texts. 
        Please refrain from adding comments and only furnish the numeric value with 2 numbers after the dot.
        DO NOT ADD any comentary like "Similatiry score: "
        
        Text 1: {text1} 
        Text 2: {text2}
        """

    messages = [{'role': "user", 'content': prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [ ]:
# Generate a xlsx file with a template for the questions
generate_df = pd.DataFrame(columns=['Questions', 'Expected_Questions'])
generate_df.to_excel(base_path, index=False)